In [6]:
import pandas as pd
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By 
from bs4 import BeautifulSoup
import time
import json 

import extracao

c:\Users\irani\Desktop\Insper Metricis\Plataforma Metricis\Dados\Projeto-TLC\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_excel("../Base 5.1.xlsx", sheet_name="Bloqueado_com_Resumo") 
df

,Nome,Núcleo,Trabalho,Data Publicação,Categoria,Categoria geal,Autores,PDF,Tipo de Arquivo/Link,Resumo
0,Sérgio Lazzarini,Metricis,https://repositorio.insper.edu.br/entities/pub...,2024,Relatório de pesquisa,Pesquisa,"coelho, daphne, teodorovicz, thomaz, fritscher...",Bloqueado,text/html; charset=utf-8,True
1,Sérgio Firpo,Estudos Raciais,https://repositorio.insper.edu.br/entities/pub...,2017,Capítulo de Livro,Livros,Sérgio Pinheiro Firpo et al.,Bloqueado,text/html; charset=utf-8,True
2,Sérgio Lazzarini,Metricis,https://repositorio.insper.edu.br/entities/pub...,2018,Capítulo de Livro,Livros,"marco antonio cesar bonomo, brito, ricardo d.,...",Bloqueado,text/html; charset=utf-8,True
3,Isabela Brandão Furtado,Saúde,https://repositorio.insper.edu.br/entities/pub...,2023,Capítulo de Livro,Livros,Isabela Brandão Furtado; Carolina Pedrosa de Melo,Bloqueado,text/html; charset=utf-8,True


In [3]:
def inicializa_driver():
    # A classe Service é usada para iniciar uma instância do Chrome Webdriver
    service = Service()

    # webdriver.ChromeOptions é usada para definir a preferência para o browser do chrome
    options = webdriver.ChromeOptions()

    # Inicia-se a instância do Chrome Webdriver com as definidas "options" e "service"
    return webdriver.Chrome(service=service, options=options)


driver = inicializa_driver()
lista = []

for url in df["Trabalho"]:
    driver.get(url)
    time.sleep(1.0)
    html = driver.page_source

    soup = BeautifulSoup(html, "html.parser")

    # Remover elementos que poluem o texto (navegação, footer, scripts etc.)
    for tag in soup(["script", "style", "noscript", "header", "footer", "nav", "svg"]):
        tag.decompose()

    text = " ".join(soup.get_text(separator=" ").split())
    data = extracao.extracao_langchain(text)
    data_dict = data.model_dump()
    lista.append(json.dumps(data_dict, ensure_ascii=False, indent=2, default=extracao.enum_encoder))
    

In [4]:
dados = [json.loads(s) for s in lista]


with open("resultados.json", "w", encoding="utf-8") as f:
    json.dump(dados, f, ensure_ascii=False, indent=2)

In [3]:
with open("block_w_resum.json", "r", encoding="utf-8") as file:
    dados = json.load(file) 

df_dados = pd.DataFrame(dados)
df_dados

,classificacao,metodologia,area_avaliada,ods_relacionada,ods_number,etapa,titulo,resumo
0,Academica,Quantitativa,Desenvolvimento Social,Trabalho decente e crescimento econômico,8,Aprimoramento,Monitoring the Governance of State-Owned Enter...,O relatório analisa os esforços do Brasil para...
1,Academica,Qualitativa,Desenvolvimento Social,Trabalho decente e crescimento econômico,8,Justificativa,"Structural change, productivity growth and tra...",O capítulo analisa o processo de industrializa...
2,Academica,Qualitativa,Desenvolvimento Social,Trabalho decente e crescimento econômico,8,Mapeamento dos Determinantes,Crédito direcionado e financiamento do desenvo...,O capítulo analisa o impacto do crédito direci...
3,Academica,Qualitativa,Saúde,Saúde e bem-estar,3,Mapeamento dos Determinantes,"Teoria da Mudança: definição, utilidade e como...",O capítulo aborda a teoria da mudança como uma...


In [5]:
df_total = pd.concat([df, df_dados], axis=1)
df_total

,Nome,Núcleo,Trabalho,Data Publicação,Categoria,Categoria geal,Autores,PDF,Tipo de Arquivo/Link,Resumo,classificacao,metodologia,area_avaliada,ods_relacionada,ods_number,etapa,titulo,resumo
0,Sérgio Lazzarini,Metricis,https://repositorio.insper.edu.br/entities/pub...,2024,Relatório de pesquisa,Pesquisa,"coelho, daphne, teodorovicz, thomaz, fritscher...",Bloqueado,text/html; charset=utf-8,True,Academica,Quantitativa,Desenvolvimento Social,Trabalho decente e crescimento econômico,8,Aprimoramento,Monitoring the Governance of State-Owned Enter...,O relatório analisa os esforços do Brasil para...
1,Sérgio Firpo,Estudos Raciais,https://repositorio.insper.edu.br/entities/pub...,2017,Capítulo de Livro,Livros,Sérgio Pinheiro Firpo et al.,Bloqueado,text/html; charset=utf-8,True,Academica,Qualitativa,Desenvolvimento Social,Trabalho decente e crescimento econômico,8,Justificativa,"Structural change, productivity growth and tra...",O capítulo analisa o processo de industrializa...
2,Sérgio Lazzarini,Metricis,https://repositorio.insper.edu.br/entities/pub...,2018,Capítulo de Livro,Livros,"marco antonio cesar bonomo, brito, ricardo d.,...",Bloqueado,text/html; charset=utf-8,True,Academica,Qualitativa,Desenvolvimento Social,Trabalho decente e crescimento econômico,8,Mapeamento dos Determinantes,Crédito direcionado e financiamento do desenvo...,O capítulo analisa o impacto do crédito direci...
3,Isabela Brandão Furtado,Saúde,https://repositorio.insper.edu.br/entities/pub...,2023,Capítulo de Livro,Livros,Isabela Brandão Furtado; Carolina Pedrosa de Melo,Bloqueado,text/html; charset=utf-8,True,Academica,Qualitativa,Saúde,Saúde e bem-estar,3,Mapeamento dos Determinantes,"Teoria da Mudança: definição, utilidade e como...",O capítulo aborda a teoria da mudança como uma...


In [13]:
df_total_limpo = df_total.drop(["PDF", "Tipo de Arquivo/Link", "Resumo"], axis=1)
df_total_limpo

,Nome,Núcleo,Trabalho,Data Publicação,Categoria,Categoria geal,Autores,classificacao,metodologia,area_avaliada,ods_relacionada,ods_number,etapa,titulo,resumo
0,Sérgio Lazzarini,Metricis,https://repositorio.insper.edu.br/entities/pub...,2024,Relatório de pesquisa,Pesquisa,"coelho, daphne, teodorovicz, thomaz, fritscher...",Academica,Quantitativa,Desenvolvimento Social,Trabalho decente e crescimento econômico,8,Aprimoramento,Monitoring the Governance of State-Owned Enter...,O relatório analisa os esforços do Brasil para...
1,Sérgio Firpo,Estudos Raciais,https://repositorio.insper.edu.br/entities/pub...,2017,Capítulo de Livro,Livros,Sérgio Pinheiro Firpo et al.,Academica,Qualitativa,Desenvolvimento Social,Trabalho decente e crescimento econômico,8,Justificativa,"Structural change, productivity growth and tra...",O capítulo analisa o processo de industrializa...
2,Sérgio Lazzarini,Metricis,https://repositorio.insper.edu.br/entities/pub...,2018,Capítulo de Livro,Livros,"marco antonio cesar bonomo, brito, ricardo d.,...",Academica,Qualitativa,Desenvolvimento Social,Trabalho decente e crescimento econômico,8,Mapeamento dos Determinantes,Crédito direcionado e financiamento do desenvo...,O capítulo analisa o impacto do crédito direci...
3,Isabela Brandão Furtado,Saúde,https://repositorio.insper.edu.br/entities/pub...,2023,Capítulo de Livro,Livros,Isabela Brandão Furtado; Carolina Pedrosa de Melo,Academica,Qualitativa,Saúde,Saúde e bem-estar,3,Mapeamento dos Determinantes,"Teoria da Mudança: definição, utilidade e como...",O capítulo aborda a teoria da mudança como uma...


In [15]:
df_total_limpo.to_excel("Resumo Repo_Insper.xlsx", index=False)